# Parallel prediction of trained keras models with dask

Pertinent links: * https://github.com/dask/dask-examples/issues/35, https://github.com/dask/distributed/issues/2333

In [32]:
#Load modules
from __future__ import print_function
import keras
import sys
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import platform
import numpy as np
import glob
import dask
import distributed

print("dask version is {}".format(dask.__version__))
print("distributed version is {}".format(distributed.__version__))
print("keras version is {}".format(keras.__version__))
print(sys.version)


dask version is 2.9.1
distributed version is 2.9.1
keras version is 2.3.1
3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [8]:
# Define a trained and saved model
def train_model():
    batch_size = 128
    num_classes = 10
    epochs = 1
    
    # input image dimensions
    img_rows, img_cols = 28, 28
    
    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model

In [9]:
def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    return x_test

In [10]:
#Train model - takes 1m on laptop
model = train_model()
model.save("MNIST.h5")



Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 70s 1ms/step - loss: 0.2006 - accuracy: 0.9384 - val_loss: 0.0536 - val_accuracy: 0.9819
Test loss: 0.053598123369785027
Test accuracy: 0.9818999767303467


# Example 0: Without dask

In [20]:
model = keras.models.load_model("MNIST.h5")
x_test = load_data()

#Compute prediction in batch loop of size 100 (slightly contrived example)
batch_array = np.split(x_test,100)

results = []
for batch in batch_array:
    prediction = model.predict_on_batch(batch)
    results.append(prediction)
results[0].shape

(100, 10)

# Example 1 LocalCluster - load data and model first (serializes model and data)

In [24]:
client=distributed.Client()
model = keras.models.load_model("MNIST.h5")
x_test = load_data()
results = []
for batch in batch_array:
    prediction = dask.delayed(model.predict_on_batch)(batch)
    results.append(prediction)
#Gather
results = dask.compute(*results)

ValueError: Tensor Tensor("dense_2/Softmax:0", shape=(?, 10), dtype=float32) is not an element of this graph.

# Example 2 LocalCluster - load data and model delayed

In [28]:
#Example 2 LocalCluster - load data and model on each worker
model = dask.delayed(keras.models.load_model)("MNIST.h5")
x_test = dask.delayed(load_data)()
results = []
for batch in batch_array:
    prediction = dask.delayed(model.predict_on_batch)(batch)
    results.append(prediction)

#Gather
results = dask.compute(*results)
results[0].shape

(100, 10)

In [38]:
from deepforest import deepforest

In [55]:
%%timeit
#Example 2 LocalCluster - load data and model on each worker
model = dask.delayed(deepforest.deepforest)(weights="/Users/ben/miniconda3/envs/crowns/lib/python3.7/site-packages/deepforest/data/NEON.h5")
results = []
batch_array = glob.glob("tests/data/*.tif")
for batch in batch_array:
    prediction = dask.delayed(model.predict_image)(batch)
    results.append(prediction)

#Gather
results = dask.compute(*results)
results[0].shape

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

Reading config file: /Users/ben/miniconda3/envs/crowns/lib/python3.7/site-packages/deepforest/data/deepforest_config.yml
A blank deepforest object created. To perform prediction, either train or load an existing model.
Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/v0.2.1 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/v0.2.1
tracking <tf.Variable 'Variable_5:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_6:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_7:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_8:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_9:0' shape=(9, 4) dtype=float32> anchors


ModuleNotFoundError: No module named 'bokeh'